# Übung - Metadaten gruppieren & auswerten 

Nachdem wir im vorherigen Abschnitt gelernt haben, wie sich offene Datensätze mit Hilfe von SPARQL gezielt durchsuchen und filtern lassen, möchten wir Dr. Amir Weber in dieser Phase seiner Untersuchung weiter unterstützen. 

Um besser einschätzen zu können, welche Datenquellen für seine Analyse relevant sind, genügt es jedoch nicht, nur einzelne Datensätze zu finden. Es interessiert ihn auch, welche Institutionen regelmäßig entsprechende Daten bereitstellen und in welchen Formaten diese verfügbar sind. Dies kann Aufschluss darüber geben, wie verlässlich und zugänglich bestimmte Datenquellen sind.

In dieser Übung werden wir daher SPARQL nutzen, um Metadaten zu gruppieren und quantitativ auszuwerten. Wir analysieren, in welchen Datenformaten diese Informationen angeboten werden, um Dr. Weber bei der technischen Einschätzung der Weiterverwendbarkeit zu unterstützen.

Diese aggregierten Informationen geben ihm wertvolle Hinweise auf die Datenqualität, Veröffentlichungsfrequenz und technische Nutzbarkeit – und legen damit die Grundlage für fundierte Entscheidungen im weiteren Verlauf seiner Analyse.

Im ersten Schritt lassen wir uns anzeigen, welche Datenbereitsteller die meisten Datensätze mit dem Begriff „Baumkataster“ im Titel veröffentlicht haben. Durch diese Gruppierung erhält Dr. Weber einen schnellen Überblick darüber, welche Institutionen/Ländern in den letzten Jahren besonders aktiv im Bereich der baumbezogenen offenen Daten waren – ein wichtiger Hinweis auf mögliche Hauptquellen für seine weitere Recherche.

In [ ]:
%endpoint https://data.europa.eu/sparql
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX pg: <http://dcat-ap.de/def/politicalGeocoding/>

SELECT ?contributorID (COUNT(DISTINCT ?datasetTitle) AS ?uniqueDatasetTitleCount)
WHERE {
  ?datasetURI a dcat:Dataset;
              dct:title ?datasetTitle;
              dcatde:contributorID ?contributorID;
              dct:modified ?modified.
  FILTER((LANG(?datasetTitle) = "" || LANG(?datasetTitle) = "de") && CONTAINS(LCASE(?datasetTitle), "baumkataster"))
  FILTER(
      CONTAINS(STR(?modified), "2015") ||
      CONTAINS(STR(?modified), "2016") ||
      CONTAINS(STR(?modified), "2017") || 
      CONTAINS(STR(?modified), "2018") || 
      CONTAINS(STR(?modified), "2019") ||
      CONTAINS(STR(?modified), "2020") ||
      CONTAINS(STR(?modified), "2021") ||
      CONTAINS(STR(?modified), "2022") ||
      CONTAINS(STR(?modified), "2023") ||
      CONTAINS(STR(?modified), "2024") ||
  	  CONTAINS(STR(?modified), "2025") 


               )
}
GROUP BY ?contributorID

Endpoint set to: https://data.europa.eu/sparql

contributorID,uniqueDatasetTitleCount
{http://dcat-ap.de/def/contributors/openNRW},1
https://offenedaten.kdvz.nrw/,1
https://offenedaten-konstanz.de/,1
http://dcat-ap.de/def/contributors/schleswigHolstein,3
http://dcat-ap.de/def/contributors/openNRW,8
http://dcat-ap.de/def/contributors/openDataBayern,6
http://dcat-ap.de/def/contributors/stadtFrankfurtAmMain,1
http://dcat-ap.de/def/contributors/mobilithek,3
https://gdi-sh.de,1
http://dcat-ap.de/def/contributors/freistaatSachsen,1


```{admonition} Erklärung des Codes
:class: hinweis, dropdown
Der COUNT Befehl in SPARQL wird verwendet, um die Anzahl der Ergebnisse zu zählen, die eine bestimmte Bedingung erfüllen. Der AS Befehl folgt dem COUNT und dient dazu, das Ergebnis der Zählung einer Variablen zuzuweisen, die dann im Ergebnis verwendet werden kann. DISTINCT gibt an, dass nur eindeutige (DISTINCT) Werte der Variable ?datasetTitle gezählt werden sollen. AS ?uniqueDatasetTitleCount  weist das Ergebnis der Zählung der Variable ?uniqueDatasetTitleCount zu. Diese Variable kann dann im Ergebnis verwendet werden, um die Anzahl der eindeutigen datasetTitle für jeden contributorID anzuzeigen. Die Bedingung (LANG(?datasetTitle) = "" || LANG(?datasetTitle) = "de") wird verwendet, um sicherzustellen, dass nur Titel (?datasetTitle) ausgewählt werden, die entweder keine Sprachinformation haben oder explizit als Deutsch ("de") gekennzeichnet sind. Die Bedingung für das Aufnehmen von Ergebnisse mit keiner Sprachinformation ist auch wichtig, da nicht alle Datenbereitsteller Information über die Sprache des Datensatzes einschließen. Die Bedingung, dass nur Datensätze als jene auf Deutsch aufgelistet werden sollen, ist vielleicht in diesem Fall überflüssig, denn das Wort "Baumkataster" sollte auch erwähnt werden, also ein deutsches Wort. Trotzdem wollen wir diese Funktion hierbei hervorheben, weil wir später davon Gebrauch machen.
```

Es zeigt sich, dass die Datenbereitsteller Open Data Bayern und Rheinland-Pfalz – zwei der digital fortschrittlichsten Bundesländer Deutschlands – mit jeweils fünf Datensätzen die größte Anzahl liefern. Dieses Ergebnis erscheint plausibel, da beide Länder in der Förderung von Open Data eine Vorreiterrolle einnehmen.

Im Anschluss interessiert sich Dr. Weber dafür, in welchen Formaten die Baumkataster-Daten vorliegen, da dies direkte Auswirkungen auf deren Weiterverwendbarkeit hat. Während offene Formate wie CSV, GeoJSON oder XML eine einfache Analyse ermöglichen, können proprietäre oder unstrukturierte Formate die Weiterverarbeitung erschweren.

Daher erweitern wir unsere SPARQL-Abfrage, um für jeden Datenbereitsteller zu ermitteln, wie viele unterschiedliche Datenformate er im Zusammenhang mit Baumkataster-Daten anbietet. Diese Auswertung hilft Dr. Weber dabei einzuschätzen, bei welchen Anbietern er mit einem breiten und technisch zugänglichen Angebot rechnen kann – ein entscheidender Aspekt für seine geplante Analyse des Baumbestands.

In [2]:
%endpoint https://data.europa.eu/sparql
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX pg: <http://dcat-ap.de/def/politicalGeocoding/>

SELECT ?contributorID (COUNT(DISTINCT ?format) AS ?formatCount)
WHERE {
  ?datasetURI a dcat:Dataset;
              dct:title ?datasetTitle;
              dcatde:contributorID ?contributorID;
              dct:modified ?modified.
  OPTIONAL { ?datasetURI dcat:catalog ?catalog. }
  FILTER((LANG(?datasetTitle) = "" || LANG(?datasetTitle) = "de") && CONTAINS(LCASE(?datasetTitle), "baumkataster"))
  FILTER(
      CONTAINS(STR(?modified), "2015") ||
      CONTAINS(STR(?modified), "2016") ||
      CONTAINS(STR(?modified), "2017") || 
      CONTAINS(STR(?modified), "2018") || 
      CONTAINS(STR(?modified), "2019") ||
      CONTAINS(STR(?modified), "2020") ||
      CONTAINS(STR(?modified), "2021") ||
      CONTAINS(STR(?modified), "2022") ||
      CONTAINS(STR(?modified), "2023") ||
      CONTAINS(STR(?modified), "2024") ||
      CONTAINS(STR(?modified), "2025")
  )

  ?datasetURI dcat:distribution ?distribution.
  ?distribution dct:format ?format.
}
GROUP BY ?contributorID


Endpoint set to: https://data.europa.eu/sparql

contributorID,formatCount
{http://dcat-ap.de/def/contributors/openNRW},2
https://offenedaten.kdvz.nrw/,3
https://offenedaten-konstanz.de/,2
http://dcat-ap.de/def/contributors/schleswigHolstein,6
http://dcat-ap.de/def/contributors/openDataBayern,21
http://dcat-ap.de/def/contributors/openNRW,29
http://dcat-ap.de/def/contributors/stadtFrankfurtAmMain,1
http://dcat-ap.de/def/contributors/mobilithek,6
https://gdi-sh.de,6
http://dcat-ap.de/def/contributors/freistaatSachsen,2


Anbei auch die Ergebnisse der aufsummierten Datenformate je Datenbereitsteller. Nicht überraschend festzustellen ist, dass NRW und Bayern die breitesten Auswahl an Datenformaten leisten. Damit wird nicht impliziert, dass alle Datensätze in jeweils jedem Format vorhanden sind. Jedoch dient das als eine Maß für Flexibilität. 

Nachdem Dr. Weber nun einen Überblick darüber erhalten hat, welche Datenbereitsteller aktiv sind und in welchen Formaten die Daten zur Verfügung stehen, möchte er sich für einen konkreten Datensatz entscheiden, um seine Analyse zu starten. Dabei ist es ihm wichtig, den regionalen Bezug zum Berlin-Brandenburg-Raum zu wahren, da seine Untersuchung vor allem auf diese Region fokussiert ist.

Er entscheidet sich daher, gezielt nach Datensätzen zu suchen, die von der Plattform „Open Data Brandenburg“ bereitgestellt werden und das Stichwort „Baumkataster“ im Titel führen. Durch diese gezielte Einschränkung kann Dr. Weber sicherstellen, dass die Daten sowohl thematisch relevant als auch geographisch passend sind – und damit eine geeignete Grundlage für die weitere Analyse des Baumbestands in der Region darstellen.

Um direkt auf die Daten zugreifen zu können, erweitern wir die Abfrage um die Zugriffs-URL (accessURL). Diese verweist auf die konkrete Stelle, an der der Datensatz heruntergeladen oder weiterverarbeitet werden kann – ein praktischer Schritt, der es Dr. Weber ermöglicht, ohne Umwege mit der Analyse zu beginnen.

In [4]:
%endpoint https://www.govdata.de/sparql
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX pg: <http://dcat-ap.de/def/politicalGeocoding/>

SELECT ?datasetURI ?datasetTitle ?datasetDescription ?accessURL
WHERE {
  ?datasetURI a dcat:Dataset;
              dct:title ?datasetTitle;
              dct:description ?datasetDescription;
              dcatde:contributorID <http://dcat-ap.de/def/contributors/openDataBrandenburg>;
              dct:modified ?modified;
              dcat:distribution ?distribution.
              
  ?distribution dcat:accessURL ?accessURL.
  
  FILTER((LANG(?datasetTitle) = "" || LANG(?datasetTitle) = "de") && CONTAINS(LCASE(?datasetTitle), "baumkataster"))
  
  FILTER(
    CONTAINS(STR(?modified), "2017") || 
    CONTAINS(STR(?modified), "2018") || 
    CONTAINS(STR(?modified), "2019") || 
    CONTAINS(STR(?modified), "2020") || 
    CONTAINS(STR(?modified), "2021") || 
    CONTAINS(STR(?modified), "2022") || 
    CONTAINS(STR(?modified), "2023") || 
    CONTAINS(STR(?modified), "2024") || 
    CONTAINS(STR(?modified), "2025")
  )
}
ORDER BY ?datasetTitle


Endpoint set to: https://www.govdata.de/sparql

datasetURI,datasetTitle,datasetDescription,accessURL
https://piveau.offene.daten-brandenburg.de/api/hub/repo/datasets/45a104e1-2a3e-4b90-9e24-dc97d34b3298,Baumkataster - Amt Beetsee Gemeinde Beetzseeheide,"Datensatz zum Baumkataster (Standort, Baumart, Baummaße) im Bereich der Gemeinde Beetzseeheide",https://www.geoportal-amt-beetzsee.de/isk/baumkataster?REQUEST=GetCapabilities&SERVICE=WFS
https://piveau.offene.daten-brandenburg.de/api/hub/repo/datasets/45a104e1-2a3e-4b90-9e24-dc97d34b3298,Baumkataster - Amt Beetsee Gemeinde Beetzseeheide,"Datensatz zum Baumkataster (Standort, Baumart, Baummaße) im Bereich der Gemeinde Beetzseeheide",https://www.geoportal-amt-beetzsee.de/katalog.php?view=show&id=236
https://piveau.offene.daten-brandenburg.de/api/hub/repo/datasets/771b3471-f63a-4de8-b930-31dde0d23417,Baumkataster - Amt Beetsee Gemeinde Beetzseeheide (WFS),"Geodienst (WFS) zum Baumkataster (Standort, Baumart, Baummaße) im Bereich der Gemeinde Beetzseeheide",https://www.geoportal-amt-beetzsee.de/katalog.php?view=show&id=236
https://piveau.offene.daten-brandenburg.de/api/hub/repo/datasets/3a14f846-13a1-47d9-874f-3843007d85b7,Baumkataster - Amt Beetsee Gemeinde Beetzseeheide (WMS),"Geodienst (WMS) zum Baumkataster (Standort, Baumart, Baummaße) im Bereich der Gemeinde Beetzseeheide",https://www.geoportal-amt-beetzsee.de/katalog.php?view=show&id=236
https://piveau.offene.daten-brandenburg.de/api/hub/repo/datasets/09653fe3-ad6b-44d6-88e2-25b8042ee43d,Baumkataster Frankfurt (Oder) - (WFS-BAUMKATASTER),Der WFS Baumkataster beinhaltet alle Bäume die im öffentlichen Raum der Stadt Frankfurt (Oder) stehen.,https://geoportal.frankfurt-oder.de/wss/service/WFS_Baumkataster/guest


Dr. Weber entscheidet sich schließlich, den Datensatz „Baumkataster Frankfurt (Oder) - (WFS-BAUMKATASTER)” genauer unter die Lupe zu nehmen, da er für seine Analyse relevante Informationen zu den Baumstandorten in der Region liefern könnte. Dieser Datensatz scheint zunächst vielversprechend, da er auf den ersten Blick sowohl geografische als auch baumspezifische Daten zu liefern scheint.

Leider stellt Dr. Weber bei der Überprüfung fest, dass sich der betreffende Datensatz nicht direkt herunterladen lässt oder die bereitgestellte URL nicht wie erwartet funktioniert.

Das zeigt: Auch wenn Metadaten vielversprechende Informationen liefern, ist es in der Praxis oft notwendig, die tatsächliche Verfügbarkeit und Zugänglichkeit der Daten manuell zu prüfen. Dr. Weber wird seine Suche daher fortsetzen müssen – eventuell mit einer breiteren geografischen Eingrenzung oder indem er nach anderen relevanten Begriffen sucht.

Da sich der ausgewählte Datensatz nicht wie gewünscht herunterladen lässt, entscheidet sich Dr. Weber dazu, seine Suche thematisch zu erweitern. Ihm ist bewusst, dass nicht alle relevanten Informationen unter dem Begriff „Baumkataster“ veröffentlicht werden. Ein verwandter Begriff, der insbesondere in städtischen Kontexten häufig auftaucht, ist „Baumpflanzungen“ – ein Thema, das ebenfalls Rückschlüsse auf den aktuellen Baumbestand sowie stadtplanerische Maßnahmen erlaubt.

Dr. Weber formuliert daher eine neue SPARQL-Abfrage, die zusätzlich nach Datensätzen sucht, in deren Titel der Begriff „Baumpflanzungen“ vorkommt. So erhofft er sich, weitere geeignete Datensätze zu identifizieren, die ihm bei der Analyse des Baumwachstums und der Nachpflanzungen im Raum Berlin-Brandenburg nützliche Hinweise liefern können.

In [7]:
%endpoint https://www.govdata.de/sparql
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX pg: <http://dcat-ap.de/def/politicalGeocoding/>

SELECT ?datasetURI ?datasetTitle ?datasetDescription ?accessURL
WHERE {
  ?datasetURI a dcat:Dataset;
              dct:title ?datasetTitle;
              dct:description ?datasetDescription;
              dct:modified ?modified;
              dcat:distribution ?distribution.

  ?distribution dcat:accessURL ?accessURL.

  FILTER(
    (LANG(?datasetTitle) = "" || LANG(?datasetTitle) = "de") &&
    (
      CONTAINS(LCASE(?datasetTitle), "baumkataster") || 
      CONTAINS(LCASE(?datasetTitle), "baumpflanzungen")
    )
  )

  FILTER(
    CONTAINS(STR(?modified), "2015") || 
    CONTAINS(STR(?modified), "2016") ||
    CONTAINS(STR(?modified), "2017") || 
    CONTAINS(STR(?modified), "2018") || 
    CONTAINS(STR(?modified), "2019") || 
    CONTAINS(STR(?modified), "2020") || 
    CONTAINS(STR(?modified), "2021") || 
    CONTAINS(STR(?modified), "2022") || 
    CONTAINS(STR(?modified), "2023") || 
    CONTAINS(STR(?modified), "2024") || 
    CONTAINS(STR(?modified), "2025")
  )
}
ORDER BY ?datasetTitle
OFFSET 140

Endpoint set to: https://www.govdata.de/sparql

datasetURI,datasetTitle,datasetDescription,accessURL
http://suche.transparenz.hamburg.de/dataset/6410cc33-7180-429c-83fd-52e8c1488e08,Straßenbaumkataster Hamburg,"Das Straßenbaumkataster umfasst die flächendeckende Darstellung der Bäume auf öffentlichen Straßenflurstücken als Grundlage der Baumkontrollen zur Verkehrssicherheit und der Planung. Wesentliche Datenfelder sind: Standort, Gattung/Art, Pflanzjahr, Kronenumfang, Stammdurchmesser, biologische Parameter und Schadensmerkmale. Die schnelle grafische Visualisierung der Baumstandorte und Hintergrundkarten leistet eine integrierte GIS-Komponente. Die Sachdaten liegen in einer SQL Server- Datenbank vor. Erfasst wird im Rahmen fortlaufender Vermessungsgänge und Aktualisierungen. Die Fortschreibungen erfolgen auch im Rahmen der vorgeschriebenen Baumkontrollen zur Verkehrssicherheit. Das - Online-Straßenbaumkataster - umfasst die flächendeckende Darstellung der Bäume auf öffentlichen Straßenflurstücken mit wesentlichen Stammattributen sowie die Daten der Hamburger Port Authority, die in einer eigenen Datenbank gespeichert werden. Jedem Baum sind die folgenden Datenfelder zugeordnet: Standort, Gattung/Art, Pflanzjahr, Kronendurchmesser und Stammumfang. Diese im Rahmen regelmäßiger Baumkontrollen erfassten Daten werden für das Internet jährlich mit Stand 1. Januar aktualisiert. Sie stehen als WMS- u. WFS-Dienst sowie im Format GML als Download zur Verfügung.",https://qs-geodienste.hamburg.de/HH_WFS_Strassenbaumkataster?service=WFS&request=DescribeFeatureType&version=1.1.0&typename=de.hh.up:strassenbaumkataster
http://suche.transparenz.hamburg.de/dataset/aea728d3-2b65-4672-9cb8-4c69d4a3c9d6,Straßenbaumkataster Hamburger Hafen,"Das Baumkataster umfasst die flächendeckende Darstellung der Bäume auf öffentlichen Straßenflurstücken als Grundlage der Baumkontrollen zur Verkehrssicherheit und der Planung. Wesentliche Datenfelder: Standort, Gattung/Art, Pflanzjahr, Vitalität (ggf.CIR-Befliegung), biologische Parameter, Schadensmerkmale. Die schnelle grafische Visualisierung der Baumstandorte und Hintergrundkarten leistet eine integrierte GIS-Komponente. Die Sachdaten liegen in einer SQL-Datenbank vor. Erfasst werden sie im Rahmen fortlaufender Vermessungsgänge und Aktualisierungen. Fortschreibung im Rahmen der Baumkontrollen zur Verkehrssicherheit.",http://metaver.de/trefferanzeige?docuuid=19676799-FEDB-4D1A-A89A-26FBA887B3F0
http://suche.transparenz.hamburg.de/dataset/aea728d3-2b65-4672-9cb8-4c69d4a3c9d6,Straßenbaumkataster Hamburger Hafen,"Das Baumkataster umfasst die flächendeckende Darstellung der Bäume auf öffentlichen Straßenflurstücken als Grundlage der Baumkontrollen zur Verkehrssicherheit und der Planung. Wesentliche Datenfelder: Standort, Gattung/Art, Pflanzjahr, Vitalität (ggf.CIR-Befliegung), biologische Parameter, Schadensmerkmale. Die schnelle grafische Visualisierung der Baumstandorte und Hintergrundkarten leistet eine integrierte GIS-Komponente. Die Sachdaten liegen in einer SQL-Datenbank vor. Erfasst werden sie im Rahmen fortlaufender Vermessungsgänge und Aktualisierungen. Fortschreibung im Rahmen der Baumkontrollen zur Verkehrssicherheit.",https://api.hamburg.de/datasets/v1/strassenbaumkataster_hafen
http://suche.transparenz.hamburg.de/dataset/aea728d3-2b65-4672-9cb8-4c69d4a3c9d6,Straßenbaumkataster Hamburger Hafen,"Das Baumkataster umfasst die flächendeckende Darstellung der Bäume auf öffentlichen Straßenflurstücken als Grundlage der Baumkontrollen zur Verkehrssicherheit und der Planung. Wesentliche Datenfelder: Standort, Gattung/Art, Pflanzjahr, Vitalität (ggf.CIR-Befliegung), biologische Parameter, Schadensmerkmale. Die schnelle grafische Visualisierung der Baumstandorte und Hintergrundkarten leistet eine integrierte GIS-Komponente. Die Sachdaten liegen in einer SQL-Datenbank vor. Erfasst werden sie im Rahmen fortlaufender Vermessungsgänge und Aktualisierungen. Fortschreibung im Rahmen der Baumkontrollen zur Verkehrssicherheit.",https://daten-hamburg.de/umwelt_klima/s

Beim weiteren Durchstöbern der Datensätze – unterstützt durch den Einsatz von OFFSET zur Paginierung der Ergebnisse – stößt Dr. Weber auf einen Datensatz mit dem Titel „Straßenbaumpflanzungen in Steglitz-Zehlendorf“, der ihm sofort ins Auge fällt. Besonders positiv: Der Datensatz steht im CSV-Format zur Verfügung, was eine einfache Weiterverarbeitung ermöglicht.

Ein Blick in die Beschreibung macht jedoch deutlich, dass es sich dabei nicht um das vollständige Baumkataster handelt, sondern um eine Teilmenge, die lediglich bereits erfolgte Neupflanzungen auflistet. Es wird erklärt, dass Pflanzungen erst nach ihrer Durchführung in das eigentliche Baumkataster übernommen werden. Die enthaltenen Einträge stammen aus verschiedenen Quellen – etwa Spendenaktionen oder bezirklichen Projekten – und umfassen zudem Angaben zur Pflege der Bäume.

Dr. Weber erkennt den Wert dieser Information, stellt aber auch fest, dass sie für eine umfassende Analyse des aktuellen Baumbestands nicht ausreicht. Der Datensatz bietet zwar Einblicke in die Entwicklung und Nachpflanzung, jedoch nicht in die vollständige aktuelle Baumverteilung im Bezirk oder gar in Berlin insgesamt. Damit bleibt der umfassende Zugriff auf ein vollständiges Kataster weiterhin ausständig, und Dr. Weber wird seine Suche weiter ausbauen müssen, um eine geeignete Datengrundlage zu finden.

Da sich viele der bisher gefundenen Datensätze entweder als unvollständig oder schwer zugänglich erwiesen haben, blickt Dr. Weber nun in Richtung Nordrhein-Westfalen (NRW). Bereits bei der vorherigen Auswertung zur Datenverfügbarkeit war ihm aufgefallen, dass NRW eine besonders hohe Anzahl an relevanten Datensätzen bereitstellt – und das in verschiedenen Formaten und mit stabiler Zugriffbarkeit.

Die gute technische Performance und die Vielzahl veröffentlichter Inhalte lassen darauf schließen, dass das Land NRW ein fortschrittliches Open-Data-Angebot betreibt, das möglicherweise besser für eine umfassende Analyse geeignet ist. Aus diesem Grund richtet Dr. Weber nun gezielt seine Suche auf Datensätze, die durch Open Data NRW bereitgestellt werden und thematisch zu seiner Untersuchung rund um Baumkataster und Baumpflanzungen passen.

In [13]:
%endpoint https://www.govdata.de/sparql
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX pg: <http://dcat-ap.de/def/politicalGeocoding/>

SELECT ?datasetURI ?datasetTitle ?datasetDescription ?accessURL
WHERE {
  ?datasetURI a dcat:Dataset;
              dct:title ?datasetTitle;
              dct:description ?datasetDescription;
              dcatde:contributorID <http://dcat-ap.de/def/contributors/openNRW>;
              dct:modified ?modified.

  OPTIONAL {
    ?datasetURI dcat:distribution ?distribution.
    ?distribution dcat:accessURL ?accessURL.
  }

  FILTER(
    (LANG(?datasetTitle) = "" || LANG(?datasetTitle) = "de") &&
    (CONTAINS(LCASE(?datasetTitle), "baumkataster")
    )
  )

  FILTER(
    CONTAINS(STR(?modified), "2017") || 
    CONTAINS(STR(?modified), "2018") || 
    CONTAINS(STR(?modified), "2019") || 
    CONTAINS(STR(?modified), "2020") || 
    CONTAINS(STR(?modified), "2021") || 
    CONTAINS(STR(?modified), "2022") || 
    CONTAINS(STR(?modified), "2023") || 
    CONTAINS(STR(?modified), "2024") || 
    CONTAINS(STR(?modified), "2025")
  )
}
ORDER BY ?datasetTitle

Endpoint set to: https://www.govdata.de/sparql

datasetURI,datasetTitle,datasetDescription,accessURL
https://open-data.dortmund.de/api/v2/catalog/datasets/baumkataster,Baumkataster,Kataster der städtischen Bäume in Dortmund.,https://open-data.dortmund.de/api/v2/catalog/datasets/baumkataster/exports/csv?use_labels=true
https://open-data.dortmund.de/api/v2/catalog/datasets/baumkataster,Baumkataster,Kataster der städtischen Bäume in Dortmund.,https://open-data.dortmund.de/api/v2/catalog/datasets/baumkataster/exports/fgb
https://open-data.dortmund.de/api/v2/catalog/datasets/baumkataster,Baumkataster,Kataster der städtischen Bäume in Dortmund.,https://open-data.dortmund.de/api/v2/catalog/datasets/baumkataster/exports/geojson
https://open-data.dortmund.de/api/v2/catalog/datasets/baumkataster,Baumkataster,Kataster der städtischen Bäume in Dortmund.,https://open-data.dortmund.de/api/v2/catalog/datasets/baumkataster/exports/gpx
https://open-data.dortmund.de/api/v2/catalog/datasets/baumkataster,Baumkataster,Kataster der städtischen Bäume in Dortmund.,https://open-data.dortmund.de/api/v2/catalog/datasets/baumkataster/exports/json
https://open-data.dortmund.de/api/v2/catalog/datasets/baumkataster,Baumkataster,Kataster der städtischen Bäume in Dortmund.,https://open-data.dortmund.de/api/v2/catalog/datasets/baumkataster/exports/jsonl
https://open-data.dortmund.de/api/v2/catalog/datasets/baumkataster,Baumkataster,Kataster der städtischen Bäume in Dortmund.,https://open-data.dortmund.de/api/v2/catalog/datasets/baumkataster/exports/jsonld
https://open-data.dortmund.de/api/v2/catalog/datasets/baumkataster,Baumkataster,Kataster der städtischen Bäume in Dortmund.,https://open-data.dortmund.de/api/v2/catalog/datasets/baumkataster/exports/kml
https://open-data.dortmund.de/api/v2/catalog/datasets/baumkataster,Baumkataster,Kataster der städtischen Bäume in Dortmund.,https://open-data.dortmund.de/api/v2/catalog/datasets/baumkataster/exports/n3
https://open-data.dortmund.de/api/v2/catalog/datasets/baumkataster,Baumkataster,Kataster der städtischen Bäume in Dortmund.,https://open-data.dortmund.de/api/v2/catalog/datasets/baumkataster/exports/ov2


**Fund eines geeigneten Datensatzes**

Nach intensiver Recherche entscheidet sich Dr. Weber schließlich für den Datensatz „Baumkataster Köln 2017“. Dieser bietet eine umfassende Beschreibung des städtischen Baumbestands und enthält nicht nur geografische Informationen, sondern auch detaillierte Angaben zur Baumart, zum Alter, zur Kronenbreite, zur Stammdicke sowie zur Wuchshöhe der Bäume.

Besonders positiv fällt Dr. Weber die gute Zugänglichkeit des Datensatzes auf - die enthaltenen Informationen sind klar strukturiert, verständlich dokumentiert und stehen in einem gängigen Format zur Verfügung. Dadurch lässt sich der Datensatz problemlos in bestehende Analyseumgebungen integrieren – ein wichtiger Vorteil für die praktische Weiterverarbeitung. Auch wenn der Datensatz noch nicht den vollständigen Baumbestand der Stadt Köln abbildet, bietet er bereits jetzt eine solide Datenbasis für eine fundierte Analyse.

Damit verfügt Dr. Weber über eine valide Grundlage, um erste Auswertungen zur Baumverteilung und -struktur in einer Großstadt vorzunehmen und mögliche Modellregionen für seine Untersuchung zu identifizieren.